# Step One - Web Scraping of Canada data from wikipedia.org

In [75]:
### install beautiful soup v4

!conda install -c conda-forge beautifulsoup4 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.0-py36_0         conda-forge
    ca-certificates: 2019.5.15-1       --> 2019.9.11-hecc5488_0 conda-forge
    certifi:         2019.6.16-py36_1  --> 2019.9.11-py36_0     conda-forg

In [76]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

html_doc = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [77]:
### retreive the web page for html_doc 

page = requests.get(html_doc)


In [78]:
### process the contents of the web page request 

soup = BeautifulSoup(page.content, 'html5lib')


In [79]:
### capture the table of Canadian postal codes from the web page 

table = soup.find('table', attrs={'class':'wikitable sortable'})


In [80]:
### capture all the rows of the table 
table_rows = table.find_all('tr')


In [81]:
###print(table_rows)

In [82]:
### copy rows of table from the web page to a new list
postal_codes = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    postal_codes.append(row)


In [83]:
### convert list to a DF with column headings

df_postal_codes = pd.DataFrame(postal_codes, columns = ["Postcode", "Borough", "Neighbourhood"]) 


In [84]:
df_postal_codes.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [85]:
### drop first row from the DF that from converting the list to a DF
df_postal_codes = df_postal_codes.drop(0)

In [86]:
### remove rows where Borough = Not assigned
df_postal_codes_refined = df_postal_codes[df_postal_codes['Borough'] != "Not assigned"]


In [87]:
### remove \n from the neightbourhood column 
### if the Neighbourhood = "Not assigned" then set it to the Borough column value

for index, row in df_postal_codes_refined.iterrows():
    
    name = row['Neighbourhood']
    name = name.strip('\n')
    #if name[name_len-2:name_len] == "\n":
    df_postal_codes_refined.at[index, 'Neighbourhood'] = name  
    
    if row['Neighbourhood'] == "Not assigned":
        df_postal_codes_refined.at[index, 'Neighbourhood'] = row['Borough']  



In [88]:
### don't need to set an index for now
###df_postal_codes_refined.set_index(['Postcode'], inplace=True)


In [89]:
print(df_postal_codes_refined.head())
print(len(df_postal_codes_refined))

  Postcode           Borough     Neighbourhood
3      M3A        North York         Parkwoods
4      M4A        North York  Victoria Village
5      M5A  Downtown Toronto      Harbourfront
6      M5A  Downtown Toronto       Regent Park
7      M6A        North York  Lawrence Heights
211


In [90]:
df_postal_codes_final = df_postal_codes_refined.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()


In [91]:

print(df_postal_codes_final.head())

  Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae


In [92]:
df_postal_codes_final.shape

(103, 3)

# Step Two - get long and lat for each location 

In [95]:

#!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
#print('Data downloaded!')


path='http://cocl.us/Geospatial_data'
df_geospatial = pd.read_csv(path)
df_geospatial.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
df_postal_codes_merged = df_postal_codes_final.merge(df_geospatial, left_on='Postcode', right_on='Postal Code')

In [97]:
df_postal_codes_merged.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [99]:
df_postal_codes_merged.drop(['Postal Code'], axis=1, inplace=True)

In [100]:
df_postal_codes_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
